## Day 21

https://adventofcode.com/2023/day/21

In [13]:
def readInput21(infile):
    with open(infile) as f:
        lines = f.read().strip().splitlines()
        grid = {}
        S = (-1,-1)
        for j,l in enumerate(lines):
            for i,v in enumerate(l):
                grid[(i,j)] = v
                if v=="S":
                    S = (i,j)
        return grid, S, len(lines), len(lines[0])

In [10]:
def part1(infile,nsteps = 6):
    grid, S, ymax, xmax = readInput21(infile)
    plots = {S}
    dirs = [(+1,0),(-1,0),(0,+1),(0,-1)]
    for _ in range(nsteps):
        plots_new = set()
        for x,y in plots:
            for dx,dy in dirs:
                P = (x+dx,y+dy)
                if P in grid.keys() and grid[P]!="#":
                    plots_new.add(P)
        plots = plots_new
    return len(plots)

In [17]:
print("Test 1:",part1("examples/example21.txt",6))
print("Part 1:",part1("AOC2023inputs/input21.txt",64))

Test 1: 16
Part 1: 3729
